In [27]:
%reload_ext autoreload
%autoreload 2
import torch
import dagrad.flex as flex
from dagrad.utils import utils

utils.set_random_seed(1)
torch.manual_seed(1)

## Struct Learn via Flex

`flex.struct_learn` is a function that requires the following 5 inputs:
- `dataset`: A data matrix with `n x d` shape, where `n` is the number of samples and `d` is the number of variables/features.

- `model`: The SEM module. We provide standard SEM implementations such as---LinearModel, LogisticModel, MLP

- `constrained_solver`: An instance of a ConstrainedSolver class. We provide implementations such as `PathFollowing` and `AugmentedLagrangian`

- `unconstrained_solver`: An instance of an UnconstrainedSolver class. We provide implementation for `GradientBasedSolver`

- `loss_fn`: Instance of a Loss class. All available losses can be found at flex/loss.py

- `dag_fn`: Instance of a DagFn class. All available DAG functions can be found at flex/dags.py

## Generate Data

In [3]:
# Generate linear data
n, d, s0, graph_type, sem_type = 1000, 20, 20, "ER", "gauss"
linear_B_true = utils.simulate_dag(d, s0, graph_type)
linear_dataset = utils.simulate_linear_sem(linear_B_true, n, sem_type)

# Generate non-linear data
n, d, s0, graph_type, sem_type = 1000, 20, 20, "ER", "mlp"
nonlinear_B_true = utils.simulate_dag(d, s0, graph_type)
nonlinear_dataset = utils.simulate_nonlinear_sem(nonlinear_B_true, n, sem_type)

# Using flex to implement NOTEARS

## linear NOTEARS

In [12]:
# Linear model
model = flex.LinearModel(d)

# Use AML to solve the constrained problem
cons_solver = flex.AugmentedLagrangian(
    num_iter=10,
    num_steps=[3e4,6e4],
    l1_coeff=0.03,
)

# Use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    optimizer=torch.optim.Adam(model.parameters(), lr=3e-4),
)

# Use MSE loss
loss_fn = flex.MSELoss()

# Use Trace of matrix exponential as DAG function
dag_fn = flex.Exp()

# Learn the DAG
W_est = flex.struct_learn(
    dataset=linear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

acc = utils.count_accuracy(linear_B_true, W_est != 0)
print("Results: ", acc)

100%|██████████| 10/10 [00:37<00:00,  3.79s/it]

Total Time: 37.8957781791687
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 20}


## nonlinear NOTEARS

In [41]:
# Nonlinear model
model = flex.MLP(dims=[d, 10, 1], activation="sigmoid", bias=True)

# Use AML to solve the constrained problem
cons_solver = flex.AugmentedLagrangian(
    num_iter=10,
    num_steps=[4e4,6e4],
    l1_coeff=0.01,
    weight_decay=0.01,
)

# Use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    optimizer=torch.optim.Adam(model.parameters(), lr=3e-4),
)

# Use MSE loss
loss_fn = flex.MSELoss()

# Use Trace of matrix exponential as DAG function
dag_fn = flex.Exp()

# Learn the DAG
W_est = flex.struct_learn(
    dataset=nonlinear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

acc = utils.count_accuracy(nonlinear_B_true, W_est != 0)
print("Results: ", acc)

/Users/seren/dagrad/dagrad/flex/modules/constrained_solvers.py:159: UserWarning: Using the first value from num_steps for the first 8 iterations
  self.vwarn(f"Using the first value from num_steps for the first {missing_steps} iterations")
100%|██████████| 10/10 [05:13<00:00, 31.32s/it]

Total Time: 313.17299032211304
Results:  {'fdr': 1.0, 'tpr': 0.0, 'fpr': 0.023529411764705882, 'shd': 24, 'nnz': 4}


# Using flex to implement DAGMA

## Linear DAGMA

In [14]:
# Linear model
model = flex.LinearModel(d)

# Use path following to solve the constrained problem
cons_solver = flex.PathFollowing(
    num_iter=5,
    mu_init=1.0,
    mu_scale=0.1,
    logdet_coeff=[1.0, .9, .8, .7, .6],
    num_steps=[3e4, 6e4],
    l1_coeff=0.03,
)

# use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    optimizer=torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.99, 0.999))
)

# Use MSE loss
loss_fn = flex.MSELoss()

# Use LogDet as DAG function
dag_fn = flex.LogDet()

W_est = flex.struct_learn(
    dataset=linear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

acc = utils.count_accuracy(linear_B_true, W_est != 0)
print("Results: ", acc)

/Users/seren/dagrad/dagrad/flex/modules/constrained_solvers.py:49: UserWarning: Using the first value from num_steps for the first 3 iterations
  self.vwarn(
100%|██████████| 5/5 [00:08<00:00,  1.77s/it]

Total Time: 8.867684841156006
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 20}


## Nonlinear DAGMA

In [29]:
# nonlinear model
model = flex.MLP(dims=[d, 10, 1], activation="sigmoid", bias=True)

# Use path following to solve the constrained problem
cons_solver = flex.PathFollowing(
    num_iter=4,
    mu_init=0.1,
    mu_scale=0.1,
    logdet_coeff=1.0,
    num_steps=[5e4, 8e4],
    weight_decay=0.02,
    l1_coeff=0.005,
)

# use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    optimizer=torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.99, 0.999))
)

# Use NLL loss
loss_fn = flex.NLLLoss()

# Use LogDet as DAG function
dag_fn = flex.LogDet()

# Learn the DAG
W_est = flex.struct_learn(
    dataset=nonlinear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

acc = utils.count_accuracy(nonlinear_B_true, W_est != 0)
print("Results: ", acc)

/Users/seren/dagrad/dagrad/flex/modules/constrained_solvers.py:51: UserWarning: Using the first value from num_steps for the first 2 iterations
  self.vwarn(
  0%|          | 0/4 [00:00<?, ?it/s]

total loss: 7.746262377447302
total loss: 7.745656129937287
total loss: 7.745049946564558
total loss: 7.744443827463069
total loss: 7.743837772766753
total loss: 7.7432317826095325
total loss: 7.742625857125301
total loss: 7.742019996447937
total loss: 7.741414200711286
total loss: 7.74080847004916
total loss: 7.740202804595346
total loss: 7.739597204483589
total loss: 7.738991669847595
total loss: 7.738386200821026
total loss: 7.737780797537501
total loss: 7.737175460130586
total loss: 7.736570188733799
total loss: 7.735964983480598
total loss: 7.7353598445043845
total loss: 7.734754771938498
total loss: 7.734149765916213
total loss: 7.733544826570736
total loss: 7.732939954035201
total loss: 7.732335148442671
total loss: 7.731730409926125
total loss: 7.731125738618471
total loss: 7.730521134652523
total loss: 7.729916598161013
total loss: 7.729312129276586
total loss: 7.728707728131785
total loss: 7.728103394859066
total loss: 7.727499129590781
total loss: 7.726894932459177
total los

  0%|          | 0/4 [00:05<?, ?it/s]

total loss: 7.301415508968701
total loss: 7.300880711501703
total loss: 7.300346011594475
total loss: 7.2998114092310225


KeyboardInterrupt: 